In [1]:
import sys,os,pandas,numpy,copy,shutil
import torch
from torch.autograd import Variable
torch.cuda.is_available()

True

In [2]:
sys.path.append('/home/heyangle/Desktop/ScanCraft/ScanCraft')
from command.scan import scan
from command.NMSSMTools import NMSSMTools
from command.NMSSMTools import ReadNMSSMToolsSpectr
from command.operations.GetFiles import GetFiles

free=scan()
free.Add('tanB','MINPAR',3,1.,60.)
free.Add('M1','EXTPAR',1  ,20.    ,1000.)
free.Add('M2'	,'EXTPAR'   ,2  ,100.    ,2000.)
free.Add('Atop'	,'EXTPAR'   ,11  ,  -6e3    ,6e3)
free.Add('Abottom','EXTPAR'   ,12,-6e3,6e3,pace='follow Atop')
free.Add('Atau'	,'EXTPAR'   ,13  ,  100.      ,2000.)
free.Add('MtauL','EXTPAR'   ,33,	100.,	2.e3,pace='follow Atau')
free.Add('MtauR','EXTPAR'   ,36,	100.,	2.e3,pace='follow Atau')
free.Add('MQ3L'	,'EXTPAR'   ,43,	100.,	2.e3)
free.Add('MtopR'	,'EXTPAR'   ,46,	100.,	2.e3)
free.Add('MbottomR','EXTPAR'  ,49,	100.,	2.e3 ,pace='follow MtopR')
free.Add('Lambda','EXTPAR'  ,61  ,1e-3    ,1. ,pace='lognormal')
free.Add('Kappa','EXTPAR'   ,62 ,1.e-3    ,1. ,pace='lognormal')
free.Add('A_Lambda','EXTPAR' ,63,-3.e3,3.e3)
free.Add('A_kappa','EXTPAR' ,64,-3.e3,3.e3)
free.Add('mu_eff','EXTPAR'  ,65,100.,1500.)

ignore=[ 'Landau Pole'#27
        ,'relic density'#30
        ,'b->s gamma'#32
        ,'B_s->mu+mu-'#35
        ,'Muon magn'#37
        ,'No Higgs in the'#46
        ,'b -> c tau nu'#58 always keep alive
        ]

free.GetValue('./Pylon/carrier/inpZ3.dat')

In [15]:
import threading,time
def warp(number):
    position='./Pylon/prob'+str(number)
    try:
        shutil.copytree('./Pylon/prob/',position)
    except FileExistsError:
        pass
    N=NMSSMTools(package_dir=position,data_dir='./Pylon/carrier',in_model='inpZ3.dat',clean=False)
    return N

ore_lock=threading.Lock()
samples_lock=threading.Lock()
samples=[]
class MT_NTools(threading.Thread):
    def __init__(self,ID,sequence,samples):
        threading.Thread.__init__(self)
        self.ID=ID
        self.N=warp(ID)
        self.sequence=sequence
        self.samples=samples
    def run(self):
        while not self.sequence.empty():
            ore_lock.acquire() # LOCK-----------
            ore=self.sequence.get()
            sys.stdout.write(' '.join([
                repr(i) for i in [self.ID,'runing',ore.variable_list['tanB'].value,'at',time.ctime()]
                ])+'\n')
            ore_lock.release()
            
            sample=self.N.Run(ore)
            
            samples_lock.acquire() # LOCK-----------
            if len(sample.SPINFO[4])==0:
                self.samples.append(sample)
                self.N.Record(len(self.samples)-1)
                sys.stdout.write(' '.join([
                    repr(i) for i in [self.ID,'done',sample.MINPAR,'at',time.ctime(),len(self.samples)]
                    ])+'\n')
            else:
                sys.stdout.write(' '.join([
                    repr(i) for i in [self.ID,'excluded',sample.MINPAR[3],'at',time.ctime(),sample.SPINFO]
                    ])+'\n')
            samples_lock.release()


In [13]:
import queue
ore_queue=queue.Queue(20)
for vt in range(20):
    new_ore=copy.deepcopy(free)
    new_ore.variable_list['tanB'].value=vt*0.1+3.
    ore_queue.put(new_ore)
NT=[]
for i in range(4):
    NT.append(MT_NTools(i,ore_queue,samples))
len(NT)
for N_i in NT:
    N_i.start()
for N_i in NT:
    N_i.join()

0 'runing' 3.0 'at' 'Sun Nov 19 14:51:21 2017'
3 'runing' 3.3 'at' 'Sun Nov 19 14:51:21 2017'
1 'runing' 3.1 'at' 'Sun Nov 19 14:51:21 2017'
2 'runing' 3.2 'at' 'Sun Nov 19 14:51:21 2017'
1 'done' {3: 3.1} 'at' 'Sun Nov 19 14:51:23 2017' 10
1 'runing' 3.4 'at' 'Sun Nov 19 14:51:23 2017'
2 'done' {3: 3.2} 'at' 'Sun Nov 19 14:51:23 2017' 11
2 'runing' 3.5 'at' 'Sun Nov 19 14:51:23 2017'
0 'done' {3: 3.0} 'at' 'Sun Nov 19 14:51:23 2017' 12
0 'runing' 3.6 'at' 'Sun Nov 19 14:51:23 2017'
3 'done' {3: 3.3} 'at' 'Sun Nov 19 14:51:23 2017' 13
3 'runing' 3.7 'at' 'Sun Nov 19 14:51:23 2017'
3 'done' {3: 3.7} 'at' 'Sun Nov 19 14:51:25 2017' 14
3 'runing' 3.8 'at' 'Sun Nov 19 14:51:25 2017'
0 'done' {3: 3.6} 'at' 'Sun Nov 19 14:51:25 2017' 15
0 'runing' 3.9 'at' 'Sun Nov 19 14:51:25 2017'
2 'done' {3: 3.5} 'at' 'Sun Nov 19 14:51:25 2017' 16
2 'runing' 4.0 'at' 'Sun Nov 19 14:51:25 2017'
0 'excluded' 3.9 'at' 'Sun Nov 19 14:51:25 2017' {1: ['NMSSMTools'], 2: ['5.2.0'], 3: [], 4: ['# M_H1^2<1']}
0 '

In [14]:
for i,sample in enumerate(samples):
    print(i,sample.__dict__.keys())

0 dict_keys(['DECAY', 'SPINFO', 'MODSEL', 'SMINPUTS', 'MINPAR', 'EXTPAR', 'MASS', 'LOWEN', 'HMIX', 'NMHMIX', 'NMAMIX', 'STOPMIX', 'SBOTMIX', 'STAUMIX', 'NMNMIX', 'UMIX', 'VMIX', 'REDCOUP', 'HIGGSBOUNDSINPUTHIGGSCOUPLINGSBOSONS', 'HIGGSBOUNDSINPUTHIGGSCOUPLINGSFERMIONS', 'GAUGE', 'YU', 'YD', 'YE', 'AU', 'TU', 'AD', 'TD', 'AE', 'TE', 'MSOFT', 'NMSSMRUN', 'MSQ2', 'MSU2', 'MSD2', 'MSL2', 'MSE2', 'USQMIX', 'DSQMIX', 'SELMIX', 'GUTGAUGE', 'GUTYU', 'GUTYD', 'GUTYE', 'GUTAU', 'GUTAD', 'GUTAE', 'GUTMSOFT', 'GUTNMSSMRUN', 'FINETUNING', 'LHCCROSSSECTIONS', 'LHCFIT', 'EFFECTIVE_COUPLINGS', 'DCINFO', 'RDINFO', 'ABUNDANCE', 'LSP', 'NDMCROSSSECT', 'ANNIHILATION', 'ERROR', 'constraints'])
1 dict_keys(['DECAY', 'SPINFO', 'MODSEL', 'SMINPUTS', 'MINPAR', 'EXTPAR', 'MASS', 'LOWEN', 'HMIX', 'NMHMIX', 'NMAMIX', 'STOPMIX', 'SBOTMIX', 'STAUMIX', 'NMNMIX', 'UMIX', 'VMIX', 'REDCOUP', 'HIGGSBOUNDSINPUTHIGGSCOUPLINGSBOSONS', 'HIGGSBOUNDSINPUTHIGGSCOUPLINGSFERMIONS', 'GAUGE', 'YU', 'YD', 'YE', 'AU', 'TU', 'AD', 'T

In [17]:
samples[0].SPINFO

{1: ['NMSSMTools'], 2: ['5.2.0'], 3: [], 4: ['# M_H1^2<1']}